In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
import time
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver


In [2]:
load_dotenv()

llm = ChatOpenAI()

In [3]:
class CrashState(TypedDict) :
    input : str
    step1 : str
    step2 : str
    step3 : str


In [4]:
# Define Steps 
def step_1(state: CrashState) :
    print("Step 1 executed")
    return {'step1': "done"}
def step_2(state: CrashState) :
    print("Step 2 hanging...now manually interpt the code")
    time.sleep(30)
    return {'step2': "done"}
def step_3(state: CrashState) :
    print("Step 3 executed")
    return {'step3': "done"}


In [5]:
# Build the graph 
builder = StateGraph(CrashState) 
builder.add_node("step_1",step_1)
builder.add_node("step_2",step_2)
builder.add_node("step_3",step_3)

builder.add_edge(START, 'step_1')
builder.add_edge('step_1', 'step_2')
builder.add_edge('step_2', 'step_3')
builder.add_edge('step_3', END)

checkpointer = InMemorySaver()

workflow = builder.compile(checkpointer=checkpointer)

In [6]:
config = {'configurable' : {'thread_id' : 1}}
workflow.invoke({'input' : 'input'}, config = config)

Step 1 executed
Step 2 hanging...now manually interpt the code


KeyboardInterrupt: 

In [7]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'input': 'input', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c120e-fa23-6c3e-8001-7d815b7cff99'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-14T06:12:45.198649+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c120e-fa1e-6e25-8000-a75445089db6'}}, tasks=(PregelTask(id='e202d428-4d2d-e335-e731-cdde009543ac', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'input'}, next=('step_1',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c120e-fa1e-6e25-8000-a75445089db6'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-11-14T06:12:45.196650+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c120e-fa1c

In [9]:
workflow.get_state(config)

StateSnapshot(values={'input': 'input', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c120e-fa23-6c3e-8001-7d815b7cff99'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-14T06:12:45.198649+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c120e-fa1e-6e25-8000-a75445089db6'}}, tasks=(PregelTask(id='e202d428-4d2d-e335-e731-cdde009543ac', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [10]:
# After manually interupting the workflow we can resume the workflow from step 2 this is the felexibility persistance gives to us 
# Here is how we can resume the workflow stored in database/ram
final_state = workflow.invoke(None, config=config)
print(f'final State : \n{final_state}')

Step 2 hanging...now manually interpt the code
Step 3 executed
final State : 
{'input': 'input', 'step1': 'done', 'step2': 'done', 'step3': 'done'}
